# Testing GPU

In [1]:
import tensorflow as tf

In [2]:
tf.config.list_physical_devices("GPU")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [1]:
import tensorflow as tf

# List all physical GPUs
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("GPUs available: ", len(gpus))
    for gpu in gpus:
        print("GPU Name: ", gpu)
else:
    print("No GPUs available")


GPUs available:  1
GPU Name:  PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


# Loading Requirements

In [2]:
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt

import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.layers import Input, LSTM, RepeatVector, TimeDistributed, Dense, Embedding, Dropout, InputLayer
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Loading Dataset

In [3]:
df = pd.read_csv("Combined Data.csv")
df.head()

,Unnamed: 0,statement,status
0,0,oh my gosh,Anxiety
1,1,"trouble sleeping, confused mind, restless hear...",Anxiety
2,2,"All wrong, back off dear, forward doubt. Stay ...",Anxiety
3,3,I've shifted my focus to something else but I'...,Anxiety
4,4,"I'm restless and restless, it's been a month n...",Anxiety


In [20]:
df = df[~df["status"].isin(["Suicidal", "Bipolar"])]
df["status"].value_counts()

status
adhd                    33392
Normal                  32702
Depression              30808
addiction                8198
Anxiety                  7776
Stress                   5338
Personality disorder     2402
Name: count, dtype: int64

In [5]:
df_addiction = pd.read_csv("addiction_pre_features_tfidf_256.csv")
df_adhd = pd.read_csv("adhd_pre_features_tfidf_256.csv")

In [6]:
df.drop(["Unnamed: 0"], axis = 1, inplace = True)

In [7]:
# df_lonely = df_lonely[["post", "subreddit"]]
df_adhd = df_adhd[["post", "subreddit"]]
df_addiction = df_addiction[["post", "subreddit"]]

In [8]:
# df_lonely.rename(columns = {'subreddit':'status', "post" : "statement"}, inplace = True)
df_adhd.rename(columns = {'subreddit':'status', "post" : "statement"}, inplace = True)
df_addiction.rename(columns = {'subreddit':'status', "post" : "statement"}, inplace = True)

In [9]:
df_combined = pd.concat([df, df_addiction, df_adhd])

# Resetting index for the combined DataFrame
df_combined.reset_index(drop=True, inplace=True)


In [10]:
df_combined["status"].value_counts()

status
adhd                    16696
Normal                  16351
Depression              15404
addiction                4099
Anxiety                  3888
Stress                   2669
Personality disorder     1201
Name: count, dtype: int64

# Text Preprocessing

In [11]:
def preprocessing(text):
    text = str(text)
    
    # Remove retweet "RT" text
    text = re.sub(r'^RT[\s]+', '', text)
    
    # Remove HTML line breaks
    text = re.sub(r'<br />', '', text)
    
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    
    # Remove special characters and numbers
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    
    # Remove single quotes
    text = text.replace("\'", "")
    
    # Tokenize text
    tokens = text.split()
    
    # Get stopwords and initialize stemmer
    stopwords_english = stopwords.words("english")
    stemmer = PorterStemmer()
    
    cleaned_words = []

    for x in tokens:
        if x not in stopwords_english:
            stem_word = stemmer.stem(x)
            cleaned_words.append(stem_word)
    
    return ' '.join(cleaned_words)


In [12]:
def augment_text(text):
    try:
        blob = TextBlob(text)
        translated = blob.translate(to='fr').translate(to='en')
        return str(translated)
    except Exception as e:
        return text

df_combined['augmented_statement'] = df_combined['statement'].apply(augment_text)
augmented_df = df_combined[['statement', 'status']].copy()
augmented_df['statement'] = df_combined['augmented_statement']
df = pd.concat([df_combined, augmented_df])

In [13]:
df.status.value_counts()

status
adhd                    33392
Normal                  32702
Depression              30808
addiction                8198
Anxiety                  7776
Stress                   5338
Personality disorder     2402
Name: count, dtype: int64

In [16]:
df_new = df[df['status'] == "Personality disorder"]
aug_df["statement"] = df_new["statement"].apply(augment_text)
aug_df["status"] = df_new["status"]
aug_df.head()

NameError: name 'aug_df' is not defined

In [14]:
df["cleaned_post"] = df["statement"].apply(preprocessing)

In [15]:
df['cleaned_post'] = df['cleaned_post'].fillna('')

# Data Spliting

In [16]:
X = df['cleaned_post']
y = df['status']

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tokenization

In [18]:
X_train = pd.Series(X_train).fillna("").astype(str).tolist()
X_test = pd.Series(X_test).fillna("").astype(str).tolist()

tokenizer = Tokenizer(num_words = 5000, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

train_x_sequences = tokenizer.texts_to_sequences(X_train)
test_x_sequences = tokenizer.texts_to_sequences(X_test)

max_length = 100
train_x_padded = pad_sequences(train_x_sequences, maxlen=max_length, padding='post', truncating='post')
test_x_padded = pad_sequences(test_x_sequences, maxlen=max_length, padding='post', truncating='post')

In [237]:
import pickle

# Save the tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [19]:
tokenizer_json = tokenizer.to_json()
with open('tokenizer.json', 'w') as json_file:
    json_file.write(tokenizer_json)

# Modeling

In [150]:
from keras.layers import Bidirectional, BatchNormalization
from keras.regularizers import l2

model = Sequential([
    Embedding(input_dim=5000, output_dim=64, input_length=max_length),
    
    Bidirectional(LSTM(64, return_sequences=True)),
    Dropout(0.6),  # Increase dropout to prevent overfitting
    
    Bidirectional(LSTM(64)),
    Dropout(0.6),
    
    BatchNormalization(),  # Adding Batch Normalization for better generalization
    
    Dense(64, activation='relu', kernel_regularizer=l2(0.01)),  # Adding L2 regularization
    Dropout(0.5),
    
    Dense(7, activation='softmax')  # Use softmax for multi-class classification
])
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 100, 64)           320000    
                                                                 
 bidirectional_21 (Bidirecti  (None, 100, 128)         66048     
 onal)                                                           
                                                                 
 dropout_21 (Dropout)        (None, 100, 128)          0         
                                                                 
 bidirectional_22 (Bidirecti  (None, 128)              98816     
 onal)                                                           
                                                                 
 dropout_22 (Dropout)        (None, 128)               0         
                                                                 
 batch_normalization_1 (Batc  (None, 128)             

In [218]:
from sklearn.preprocessing import LabelEncoder

labels = ['adhd', 'Normal', 'Depression', 'addiction', 'Anxiety', 'Stress', 'Personality disorder']

label_encoder = LabelEncoder()

train_y_encoded = label_encoder.fit_transform(y_train)
test_y_encoded = label_encoder.transform(y_test)
label_mapping = dict(zip(label_encoder.classes_, range(len(label_encoder.classes_))))
print(label_mapping)

{'Anxiety': 0, 'Depression': 1, 'Normal': 2, 'Personality disorder': 3, 'Stress': 4, 'addiction': 5, 'adhd': 6}


In [26]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1)
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.0001, verbose=1)

In [34]:
df.status.value_counts()

status
Normal                  32702
Depression              30808
addiction                8198
Anxiety                  7776
Bipolar                  5754
Stress                   5338
Personality disorder     2402
Name: count, dtype: int64

In [113]:
import numpy as np

class_counts = np.array([7776, 5754, 30808, 32702, 2402, 5338, 8198])
total_samples = np.sum(class_counts)
num_classes = len(class_counts)
class_weights = {i: total_samples / (num_classes * count) for i, count in enumerate(class_counts)}

print(class_weights)

{0: 2.1356310013717423, 1: 2.886108214575368, 2: 0.5390374794425691, 3: 0.5078180743277679, 4: 6.91368304190952, 5: 3.1110278506306983, 6: 2.0256973245507033, 7: 0.4973247085130171, 8: 0.7192146672441172}


In [151]:
history = model.fit(train_x_padded,
                           train_y_encoded,
                           epochs=10,
                           batch_size=128,
                           validation_data=(test_x_padded, test_y_encoded),
                           class_weight=class_weights,
                           callbacks=[early_stopping, model_checkpoint, reduce_lr]
                          )

Epoch 1/10
934/935 [============================>.] - ETA: 0s - loss: 1.4488 - accuracy: 0.6322
Epoch 1: val_accuracy did not improve from 0.89827
935/935 [==============================] - 63s 63ms/step - loss: 1.4489 - accuracy: 0.6322 - val_loss: 0.8660 - val_accuracy: 0.7692 - lr: 0.0010
Epoch 2/10
934/935 [============================>.] - ETA: 0s - loss: 0.8386 - accuracy: 0.7969
Epoch 2: val_accuracy did not improve from 0.89827
935/935 [==============================] - 55s 59ms/step - loss: 0.8386 - accuracy: 0.7969 - val_loss: 0.6470 - val_accuracy: 0.8266 - lr: 0.0010
Epoch 3/10
934/935 [============================>.] - ETA: 0s - loss: 0.6814 - accuracy: 0.8344
Epoch 3: val_accuracy did not improve from 0.89827
935/935 [==============================] - 54s 57ms/step - loss: 0.6814 - accuracy: 0.8344 - val_loss: 0.6204 - val_accuracy: 0.8357 - lr: 0.0010
Epoch 4/10
934/935 [============================>.] - ETA: 0s - loss: 0.6051 - accuracy: 0.8512
Epoch 4: val_accuracy did

In [152]:
sample_text = [df[df["status"] == "Personality disorder"].iloc[88]["cleaned_post"]]
sample_sequences = tokenizer.texts_to_sequences(sample_text)
sample_padded = pad_sequences(sample_sequences, maxlen=max_length, padding='post', truncating='post')

predictions = model.predict(sample_padded)
label_encoder.inverse_transform([np.argmax(predictions)])

1/1 [==============================] - 1s 1s/step


array(['Personality disorder'], dtype=object)

In [153]:
sample_text = ["Every day feels like a battle with an invisible enemy. My thoughts race, my heart pounds, and I can't seem to find peace, no matter how hard I try. Simple tasks feel overwhelming, and I constantly worry about things that might never happen. It's exhausting, and I long for calm."]
sample_sequences = tokenizer.texts_to_sequences(sample_text)
sample_padded = pad_sequences(sample_sequences, maxlen=max_length, padding='post', truncating='post')

predictions = model.predict(sample_padded)
label_encoder.inverse_transform([np.argmax(predictions)])

1/1 [==============================] - 0s 41ms/step


array(['Bipolar'], dtype=object)

In [154]:
sample_text = ["My moods shift like the changing seasons, from highs where I feel unstoppable, full of ideas and energy, to lows where even getting out of bed feels impossible. It's confusing and tiring, not knowing which version of me will wake up each day. I just want stability."]
sample_sequences = tokenizer.texts_to_sequences(sample_text)
sample_padded = pad_sequences(sample_sequences, maxlen=max_length, padding='post', truncating='post')

predictions = model.predict(sample_padded)
label_encoder.inverse_transform([np.argmax(predictions)])

1/1 [==============================] - 0s 42ms/step


array(['Bipolar'], dtype=object)

In [155]:
sample_text = ["It's like a heavy fog has settled over my life, dulling everything. I feel disconnected from the world, and nothing seems to bring joy anymore. Even the things I used to love don't matter now. It's hard to find the motivation to do anything, even the basics."]
sample_sequences = tokenizer.texts_to_sequences(sample_text)
sample_padded = pad_sequences(sample_sequences, maxlen=max_length, padding='post', truncating='post')

predictions = model.predict(sample_padded)
label_encoder.inverse_transform([np.argmax(predictions)])

1/1 [==============================] - 0s 41ms/step


array(['addiction'], dtype=object)

In [156]:
sample_text = ["Life has its ups and downs, but overall, things feel manageable. I have my moments of stress and sadness, but I also find joy and fulfillment in the little things. I try to stay balanced, focusing on the positive and working through challenges as they come"]
sample_sequences = tokenizer.texts_to_sequences(sample_text)
sample_padded = pad_sequences(sample_sequences, maxlen=max_length, padding='post', truncating='post')

predictions = model.predict(sample_padded)
label_encoder.inverse_transform([np.argmax(predictions)])

1/1 [==============================] - 0s 41ms/step


array(['Stress'], dtype=object)

In [157]:
sample_text = ["I often feel like I'm wearing a mask, unsure of who I really am. My relationships are complicated, and sometimes, I react in ways that even I don't understand. It's like I'm constantly trying to figure out how to fit in, but never quite finding my place."]
sample_sequences = tokenizer.texts_to_sequences(sample_text)
sample_padded = pad_sequences(sample_sequences, maxlen=max_length, padding='post', truncating='post')

predictions = model.predict(sample_padded)
label_encoder.inverse_transform([np.argmax(predictions)])

1/1 [==============================] - 0s 41ms/step


array(['adhd'], dtype=object)

In [158]:
sample_text = ["The pressure never seems to let up. There's always something that needs to be done, and I feel like I'm constantly racing against the clock. My mind is always on, juggling responsibilities, and it’s hard to relax. Sometimes, I wish I could just pause everything."]
sample_sequences = tokenizer.texts_to_sequences(sample_text)
sample_padded = pad_sequences(sample_sequences, maxlen=max_length, padding='post', truncating='post')

predictions = model.predict(sample_padded)
label_encoder.inverse_transform([np.argmax(predictions)])

1/1 [==============================] - 0s 41ms/step


array(['adhd'], dtype=object)

In [159]:
sample_text = ["It started out as something small, just a way to cope, but now it feels like it has taken over my life. I crave it even when I don't want to, and it's affecting everything—my relationships, my work, my health. I wish I could break free."]
sample_sequences = tokenizer.texts_to_sequences(sample_text)
sample_padded = pad_sequences(sample_sequences, maxlen=max_length, padding='post', truncating='post')

predictions = model.predict(sample_padded)
label_encoder.inverse_transform([np.argmax(predictions)])

1/1 [==============================] - 0s 40ms/step


array(['addiction'], dtype=object)

In [160]:
sample_text = ["My mind is always buzzing with thoughts, ideas, and distractions. It's hard to focus on one thing for too long, and I often feel restless, like I need to keep moving. Tasks that seem simple to others feel like mountains to me. I just want to feel in control"]
sample_sequences = tokenizer.texts_to_sequences(sample_text)
sample_padded = pad_sequences(sample_sequences, maxlen=max_length, padding='post', truncating='post')

predictions = model.predict(sample_padded)
label_encoder.inverse_transform([np.argmax(predictions)])

1/1 [==============================] - 0s 41ms/step


array(['Anxiety'], dtype=object)

In [161]:
sample_text = ["Even when I'm surrounded by people, I feel a deep sense of isolation. It's like there's an invisible wall between me and everyone else. I long for connection, for someone who truly understands, but it feels like no one really sees me. The emptiness is overwhelming"]
sample_sequences = tokenizer.texts_to_sequences(sample_text)
sample_padded = pad_sequences(sample_sequences, maxlen=max_length, padding='post', truncating='post')

predictions = model.predict(sample_padded)
label_encoder.inverse_transform([np.argmax(predictions)])

1/1 [==============================] - 0s 39ms/step


array(['Depression'], dtype=object)

In [162]:
y_pred = model.predict(test_x_padded)
y_pred = [np.argmax(x) for x in y_pred]

935/935 [==============================] - 17s 18ms/step


In [163]:
from sklearn.metrics import classification_report
print("Accuracy Score:")
print(accuracy_score(test_y_encoded, y_pred))

print("Classification Report:")
print(classification_report(test_y_encoded, y_pred))

Accuracy Score:
0.8936170212765957
Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.89      0.88      1559
           1       0.83      0.87      0.85      1170
           2       0.90      0.87      0.89      6137
           3       0.95      0.92      0.93      6452
           4       0.69      0.90      0.78       512
           5       0.70      0.82      0.75      1149
           6       0.85      0.94      0.89      1681
           7       0.96      0.90      0.93      6662
           8       0.86      0.89      0.88      4570

    accuracy                           0.89     29892
   macro avg       0.84      0.89      0.86     29892
weighted avg       0.90      0.89      0.89     29892



In [164]:
model.save('RNN model v3.h5')

In [110]:
label_encoder.classes_

array(['Anxiety', 'Bipolar', 'Depression', 'Normal',
       'Personality disorder', 'Stress', 'Suicidal'], dtype=object)

In [86]:
loaded_model = load_model('RNN model.h5')

In [32]:
# sample_text = [df[df["status"] == "Personality disorder"].iloc[88]["cleaned_post"]]
sample_text = ["I hate myself  my neighbour  everyone"]
sample_sequences = tokenizer.texts_to_sequences(sample_text)
sample_padded = pad_sequences(sample_sequences, maxlen=max_length, padding='post', truncating='post')

predictions = model.predict(sample_padded)
label_encoder.inverse_transform([np.argmax(predictions)])

1/1 [==============================] - 0s 51ms/step


array(['Normal'], dtype=object)

In [221]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from keras.regularizers import l2

# Building a CNN model
model = Sequential([
    Embedding(input_dim=5000, output_dim=64, input_length=max_length),
    
    
    # Convolutional layers with activation
    Conv1D(256, kernel_size=5, activation='relu'),
    MaxPooling1D(pool_size=2),
    Dropout(0.5),

    Conv1D(128, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Dropout(0.4),
    
    # Flatten the output to feed into dense layers
    Flatten(),

    BatchNormalization(),

    Dense(64, activation='relu', kernel_regularizer=l2(0.01)),
    Dropout(0.5),

    Dense(7, activation='softmax')  # Use softmax for multi-class classification
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_15 (Embedding)    (None, 100, 64)           320000    
                                                                 
 conv1d_36 (Conv1D)          (None, 96, 256)           82176     
                                                                 
 max_pooling1d_36 (MaxPoolin  (None, 48, 256)          0         
 g1D)                                                            
                                                                 
 dropout_52 (Dropout)        (None, 48, 256)           0         
                                                                 
 conv1d_37 (Conv1D)          (None, 46, 128)           98432     
                                                                 
 max_pooling1d_37 (MaxPoolin  (None, 23, 128)          0         
 g1D)                                                

In [125]:
import numpy as np

class_counts = np.array([7776, 5754, 30808, 32702, 2402, 5338, 8198])
total_samples = np.sum(class_counts)
num_classes = len(class_counts)
class_weights = {i: total_samples / (num_classes * count) for i, count in enumerate(class_counts)}

print(class_weights)

{0: 1.70814961787184, 1: 2.3084065743085556, 2: 0.43114033460696666, 3: 0.4061700027084407, 4: 5.529796598073035, 5: 2.4883048760905635, 6: 1.620220959815983}


In [222]:
history = model.fit(train_x_padded,
                           train_y_encoded,
                           epochs=10,
                           batch_size=256,
                           validation_data=(test_x_padded, test_y_encoded),
#                            class_weight=class_weights,
                          )

Epoch 1/10
377/377 [==============================] - 8s 18ms/step - loss: 1.0934 - accuracy: 0.7196 - val_loss: 0.8840 - val_accuracy: 0.8243
Epoch 2/10
377/377 [==============================] - 7s 17ms/step - loss: 0.5591 - accuracy: 0.8598 - val_loss: 0.4532 - val_accuracy: 0.8837
Epoch 3/10
377/377 [==============================] - 7s 17ms/step - loss: 0.4662 - accuracy: 0.8892 - val_loss: 0.4135 - val_accuracy: 0.9036
Epoch 4/10
377/377 [==============================] - 7s 17ms/step - loss: 0.4137 - accuracy: 0.9077 - val_loss: 0.3962 - val_accuracy: 0.9119
Epoch 5/10
377/377 [==============================] - 7s 17ms/step - loss: 0.3681 - accuracy: 0.9217 - val_loss: 0.3557 - val_accuracy: 0.9242
Epoch 6/10
377/377 [==============================] - 7s 17ms/step - loss: 0.3376 - accuracy: 0.9309 - val_loss: 0.3290 - val_accuracy: 0.9327
Epoch 7/10
377/377 [==============================] - 7s 18ms/step - loss: 0.2990 - accuracy: 0.9416 - val_loss: 0.3288 - val_accuracy: 0.9356

In [223]:
y_pred = model.predict(test_x_padded)
y_pred = [np.argmax(x) for x in y_pred]

754/754 [==============================] - 2s 2ms/step


In [224]:
from sklearn.metrics import classification_report
print("Accuracy Score:")
print(accuracy_score(test_y_encoded, y_pred))

print("Classification Report:")
print(classification_report(test_y_encoded, y_pred))

Accuracy Score:
0.9500497429945283
Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.91      0.91      1551
           1       0.95      0.96      0.96      6174
           2       0.95      0.98      0.96      6589
           3       0.83      0.76      0.79       476
           4       0.90      0.70      0.79      1086
           5       0.95      0.96      0.95      1623
           6       0.98      0.97      0.97      6625

    accuracy                           0.95     24124
   macro avg       0.92      0.89      0.91     24124
weighted avg       0.95      0.95      0.95     24124



In [236]:
model.save('CNN model v3.h5')

In [182]:
model = load_model('CNN model v1.h5')

# Anxiety 

In [228]:
sample_text = ["Every day feels like a battle with my own thoughts. My heart races, and I feel a constant sense of dread. Even simple tasks feel overwhelming. I can't shake the fear that something terrible is about to happen, even when I know it's irrational. I just want peace."]
sample_sequences = tokenizer.texts_to_sequences(sample_text)
sample_padded = pad_sequences(sample_sequences, maxlen=max_length, padding='post', truncating='post')

predictions = model.predict(sample_padded)
label_encoder.inverse_transform([np.argmax(predictions)])

1/1 [==============================] - 0s 18ms/step


array(['Anxiety'], dtype=object)

# adhd

In [227]:
sample_text = ["I had another day of impulsive decisions. I bought things I didn’t need and started projects I couldn’t finish. I want to have a plan, but sticking to one feels impossible. It’s exhausting to constantly fight against the urge to do a million things at once."]
sample_sequences = tokenizer.texts_to_sequences(sample_text)
sample_padded = pad_sequences(sample_sequences, maxlen=max_length, padding='post', truncating='post')

predictions = model.predict(sample_padded)
label_encoder.inverse_transform([np.argmax(predictions)])

1/1 [==============================] - 0s 19ms/step


array(['adhd'], dtype=object)

# Depression

In [229]:
sample_text = ["Today, like most days, I feel numb. There’s a heavy weight pressing down on me, making everything seem pointless. I’ve lost interest in things I used to love. Getting through the day feels like wading through thick mud. I just want to feel okay again."]
sample_sequences = tokenizer.texts_to_sequences(sample_text)
sample_padded = pad_sequences(sample_sequences, maxlen=max_length, padding='post', truncating='post')

predictions = model.predict(sample_padded)
label_encoder.inverse_transform([np.argmax(predictions)])

1/1 [==============================] - 0s 18ms/step


array(['Depression'], dtype=object)

# Normal

In [235]:
sample_text = ["Woke up feeling refreshed today. Had a good breakfast and went for a walk in the park. Work was productive, and I felt good about my progress. Spent the evening watching a movie with friends. Nothing too special, but I enjoyed the day. Life feels steady right now."]
sample_sequences = tokenizer.texts_to_sequences(sample_text)
sample_padded = pad_sequences(sample_sequences, maxlen=max_length, padding='post', truncating='post')

predictions = model.predict(sample_padded)
label_encoder.inverse_transform([np.argmax(predictions)])

1/1 [==============================] - 0s 18ms/step


array(['Normal'], dtype=object)

# Personality Disorder

In [231]:
sample_text = ["I feel like I’m constantly at war with myself. My thoughts are chaotic, and I struggle to understand who I really am. My emotions change quickly, leaving me feeling unstable and disconnected from others. I crave relationships but often push people away without meaning to."]
sample_sequences = tokenizer.texts_to_sequences(sample_text)
sample_padded = pad_sequences(sample_sequences, maxlen=max_length, padding='post', truncating='post')

predictions = model.predict(sample_padded)
label_encoder.inverse_transform([np.argmax(predictions)])

1/1 [==============================] - 0s 17ms/step


array(['Normal'], dtype=object)

# Stress

In [232]:
sample_text = ["Today, my chest felt tight all day. I’m overwhelmed by all the things I have to do and not enough time to do them. My thoughts are scattered, and I can't focus on one task. I wish I could find a way to relax, but the stress is relentless."]
sample_sequences = tokenizer.texts_to_sequences(sample_text)
sample_padded = pad_sequences(sample_sequences, maxlen=max_length, padding='post', truncating='post')

predictions = model.predict(sample_padded)
label_encoder.inverse_transform([np.argmax(predictions)])

1/1 [==============================] - 0s 17ms/step


array(['Stress'], dtype=object)

# Addiction

In [233]:
sample_text = ["I tried to stay clean today, but the cravings were too strong. I feel trapped in a cycle that I can't escape. I know it’s hurting me, but I can't stop. Each time I use, I feel a momentary relief, followed by crushing guilt and shame."]
sample_sequences = tokenizer.texts_to_sequences(sample_text)
sample_padded = pad_sequences(sample_sequences, maxlen=max_length, padding='post', truncating='post')

predictions = model.predict(sample_padded)
label_encoder.inverse_transform([np.argmax(predictions)])

1/1 [==============================] - 0s 17ms/step


array(['addiction'], dtype=object)

# 3 Normal

In [234]:
sample_text = ["Woke up feeling refreshed today. Had a good breakfast and went for a walk in the park. Work was productive, and I felt good about my progress. Spent the evening watching a movie with friends. Nothing too special, but I enjoyed the day. Life feels steady right now."]
sample_sequences = tokenizer.texts_to_sequences(sample_text)
sample_padded = pad_sequences(sample_sequences, maxlen=max_length, padding='post', truncating='post')

predictions = model.predict(sample_padded)
label_encoder.inverse_transform([np.argmax(predictions)])

1/1 [==============================] - 0s 17ms/step


array(['Normal'], dtype=object)

In [180]:
sample_text = ["Today was a typical day. Went through my routine at work, met all my deadlines. Came home, made dinner, and relaxed with a book. I’m feeling alright, just content. There’s a sense of calm in the ordinary. Sometimes, I wonder if I should be doing more."]
sample_sequences = tokenizer.texts_to_sequences(sample_text)
sample_padded = pad_sequences(sample_sequences, maxlen=max_length, padding='post', truncating='post')

predictions = model.predict(sample_padded)
label_encoder.inverse_transform([np.argmax(predictions)])

1/1 [==============================] - 0s 19ms/step


array(['Normal'], dtype=object)

In [181]:
sample_text = ["I spent the day doing a bit of everything—work, chores, a little bit of reading. It’s these kinds of days that remind me how nice it is to just have balance. Nothing extraordinary, but there’s comfort in routine. Life feels balanced, and I'm okay with that."]
sample_sequences = tokenizer.texts_to_sequences(sample_text)
sample_padded = pad_sequences(sample_sequences, maxlen=max_length, padding='post', truncating='post')

predictions = model.predict(sample_padded)
label_encoder.inverse_transform([np.argmax(predictions)])

1/1 [==============================] - 0s 18ms/step


array(['Normal'], dtype=object)